# BLEND053
- Add keras model

In [1]:
import pandas as pd
sch_oof = pd.read_csv('../oof/SchNet_ensemble_v5_oof.csv')
sch_pred = pd.read_csv('../submissions/SchNet_ensemble_v5.csv')

In [2]:
sch_oof = sch_oof.rename(columns={'scalar_coupling_constant':'SCH'})
sch_pred = sch_pred.rename(columns={'scalar_coupling_constant':'SCH'})

In [3]:
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error

def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

def log_mean_absolute_error(y_true, y_pred):
    return np.log(mean_absolute_error(y_true, y_pred))
%matplotlib inline

## Score

In [6]:
for i, d in final_oof.groupby('type'):
    score = mean_absolute_error(d['scalar_coupling_constant'], d['oof_blend'])
    lscore = np.log(mean_absolute_error(d['scalar_coupling_constant'], d['oof_blend']))
    print(i,'\t {:0.5f} \t {:0.5f}'.format(score, lscore))
glmae = group_mean_log_mae(final_oof['scalar_coupling_constant'], final_oof['oof_blend'], final_oof['type'])
print('\nGroup LMAE')
print('{:0.5f}'.format(glmae))

1JHC 	 0.29257 	 -1.22905
1JHN 	 0.16106 	 -1.82597
2JHC 	 0.10915 	 -2.21499
2JHH 	 0.06845 	 -2.68168
2JHN 	 0.07752 	 -2.55723
3JHC 	 0.12153 	 -2.10756
3JHH 	 0.07847 	 -2.54505
3JHN 	 0.06500 	 -2.73340

Group LMAE
-2.23687


In [4]:
sub = pd.read_csv('../submissions/BLEND051_sub_-2.03874CV.csv')

In [10]:
sub['scalar_coupling_constant'] = (1 * sub['scalar_coupling_constant']) + (9 * sch_pred['SCH']) / 10

In [11]:
sub.to_csv('../submissions/BLEND053.csv', index=False)

# Validate Sub

In [8]:
sub_good = pd.read_csv('../submissions/BLEND044_sub_-2.03355CV.csv')
final_sub['good_scc'] = sub_good['scalar_coupling_constant']
final_sub[['scalar_coupling_constant','good_scc']].corr()

,scalar_coupling_constant,good_scc
scalar_coupling_constant,1.0,1.0
good_scc,1.0,1.0


In [9]:
print('MAE vs Good Sub')
for i, d in final_sub.groupby('type'):
    mae_t = mean_absolute_error(d['good_scc'], d['scalar_coupling_constant'])
    corr_t = np.corrcoef(d['good_scc'], d['scalar_coupling_constant'])[1][0]
    print(f'{i} - {mae_t:0.5f} - {corr_t:0.5f}')

MAE vs Good Sub
1JHC - 0.00886 - 1.00000
1JHN - 0.02600 - 0.99999
2JHC - 0.00899 - 0.99999
2JHH - 0.01232 - 0.99999
2JHN - 0.00865 - 0.99999
3JHC - 0.00654 - 0.99999
3JHH - 0.00997 - 0.99999
3JHN - 0.00476 - 0.99998


# Create Submission and OOF Files

In [10]:
import json
import os.path
import re
import ipykernel
import requests

#try:  # Python 3
#    from urllib.parse import urljoin
#except ImportError:  # Python 2
#    from urlparse import urljoin

# Alternative that works for both Python 2 and 3:
from requests.compat import urljoin

try:  # Python 3 (see Edit2 below for why this may not work in Python 2)
    from notebook.notebookapp import list_running_servers
except ImportError:  # Python 2
    import warnings
    from IPython.utils.shimmodule import ShimWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=ShimWarning)
        from IPython.html.notebookapp import list_running_servers


def get_notebook_name():
    """
    Return the full path of the jupyter notebook.
    """
    kernel_id = re.search('kernel-(.*).json',
                          ipykernel.connect.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        response = requests.get(urljoin(ss['url'], 'api/sessions'),
                                params={'token': ss.get('token', '')})
        for nn in json.loads(response.text):
            if nn['kernel']['id'] == kernel_id:
                relative_path = nn['notebook']['path']
                return os.path.join(ss['notebook_dir'], relative_path)

In [12]:
BLEND_NUMBER = get_notebook_name().split('/')[-1].replace('.ipynb','').replace('-','')
print(f'NAME TO SAVE {BLEND_NUMBER}')

NAME TO SAVE BLEND051


In [13]:
# Save the Results
final_sub[['id','scalar_coupling_constant']].to_csv(f'../submissions/{BLEND_NUMBER}_sub_{glmae:0.5f}CV.csv', index=False)
final_oof.to_csv(f'../oof/{BLEND_NUMBER}_oof_{glmae:0.5f}CV.csv', index=False)